**IMPORTING THE DATA SET**

In [1]:
import numpy as pd
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_yelp = pd.read_csv("yelp_labelled.txt" , sep ="\t" , header = None)

In [3]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
#assign column name
column_name = ['Review' , 'Sentiment']
data_yelp.columns = column_name

In [5]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
data_yelp.shape

(1000, 2)

In [7]:
data_amazon = pd.read_csv("amazon_cells_labelled.txt" , sep = '\t' , header= None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
data_amazon.columns = column_name

In [9]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [10]:
data_amazon.shape

(1000, 2)

In [11]:
data_imdb = pd.read_csv("imdb_labelled.txt" , sep = '\t' , header= None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [12]:
data_imdb.columns = column_name

In [13]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [14]:
data_imdb.shape

(748, 2)

**WE WANT TO COMBINE ALL THE DATASET**

In [15]:
data = pd.concat([data_amazon , data_imdb , data_yelp], ignore_index = True)

In [16]:
data.shape

(2748, 2)

In [17]:
data.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [18]:
data['Sentiment'].value_counts()

,count
Sentiment,
1,1386
0,1362


In [19]:
data.isnull().sum()

,0
Review,0
Sentiment,0


**SET THE INPUT AND OUTPUT DATASET**

In [20]:
x = data['Review']
y = data['Sentiment']

**2)DATA CLEANING**

In [21]:
import string

In [22]:
punct = string.punctuation

In [23]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [25]:
import spacy

In [26]:
from spacy.lang.en.stop_words import STOP_WORDS

In [27]:
nlp = spacy.load("en_core_web_sm")

In [28]:
stopwords = list(STOP_WORDS)

In [29]:
#create the function for data cleaning
def text_data_cleaning(sentence): #its going to accept only one review each time
    doc =nlp(sentence) #we apply the spacy model to those sentences


    tokens = []
    for token in doc:
      if token.lemma_ != "-PRON-":
        temp = token.lemma_.lower().strip()
      else :
        temp = token.lower_
      tokens.append(temp)


    cleaned_token =[]
    for token in tokens:
      if token not in stopwords and token not in punct:
        cleaned_token.append(token)
    return cleaned_token



In [30]:
text_data_cleaning("Hello all , It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

**VECTORIZATION FEATURE ENGINEERING TF,IDF**




In [32]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [35]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)

In [49]:
classifier = LinearSVC()

**TRAIN THE MODEL**

In [45]:
from sklearn.model_selection import train_test_split
x_train ,x_test , y_train , y_test = train_test_split(x,y,test_size = 0.2 , random_state = 0)

In [46]:
x_train.shape , x_test.shape

((2198,), (550,))

In [47]:
x_train.head()

,Review
2572,Awful service.
526,"horrible, had to switch 3 times."
1509,"The fat computer geek was unbelievable, the bible thumper, the bad-ass girl, who are these actors???"
144,The phone takes FOREVER to charge like 2 to 5 hours literally.
2483,"Probably never coming back, and wouldn't recommend it."


**FIT THE X_TRAIN AND Y_TRAIN**

In [50]:
clf = Pipeline([('tfidf',tfidf),('clf', classifier)])

In [51]:
clf.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7a48cf96b910>)),
                ('clf', LinearSVC())])

**PREDICT THE TEST SET RESULT**

In [54]:
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix


In [53]:
y_pred = clf.predict(x_test)

In [55]:
confusion_matrix (y_test , y_pred)

array([[221,  60],
       [ 58, 211]])

In [56]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79       281
           1       0.78      0.78      0.78       269

    accuracy                           0.79       550
   macro avg       0.79      0.79      0.79       550
weighted avg       0.79      0.79      0.79       550



In [57]:
accuracy_score(y_test ,y_pred)

0.7854545454545454

In [59]:
clf.predict(['Wow, I am learning Natural Language Processing in fun fashion!'])
#the review is positive

array([1])

In [60]:
clf.predict(["It's hard to learn new things!"])
#that means the review is negative

array([0])